# Get required imports

In [1]:
import socket
import random

# First Layer the User Datagram Protocol.
    - Constructor: The user assigns a channel to bind with the server which will send the informaton through.
    - recieveMessage(): Listens and recieves any message that arrives in the channel.
    - sendMessage(): Sends through the channel a response that is meant as a reply from the recieved message.
    - closeConnection(): Shuts down the server.

In [2]:
class UDPServer:

    def __init__(self, server_address, debug):
        self.sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
        self.debug = debug
        if self.debug:
            print('DEBUG - Starting up on {} port {}'.format(*server_address))
        self.sock.bind(server_address)
        self.server_address = server_address

    def recieveMessage(self):
        if self.debug:
            print('DEBUG - waiting to receive message')
        data, self.server_address = self.sock.recvfrom(4096)
        if self.debug:
            print('DEBUG - received {} bytes from {}'.format(len(data), self.server_address))
        return data.decode()

    def sendMessage(self, data):
        if random.randint(0, 5) == 0:
            if self.debug:
                print('\nDEBUG - Data did not send!')
            return False
        if random.randint(0, 10) == 0:
            self.sock.sendto(str.encode(str(data)), self.server_address)
        sent = self.sock.sendto(str.encode(str(data)), self.server_address)
        if self.debug:
            print('DEBUG - sent {} bytes back to {}'.format(sent, self.server_address))

    def closeConnection(self):
        if self.debug:
            print('DEBUG - closing socket')
        self.sock.close()

# Second Layer the Stop and Wait Protocol.
    - Constructor: The user assigns a channel to send messages back to the client.
    - recieveMessage(): Listens and recieves any message that arrives in the channel, sending a reply immidiatly and sending any new info into the server.
    - closeConnection(): Shuts down the server.

In [3]:
class SAWServer:

    def __init__(self, ip_address, debug):
        self.debug = debug
        self.conn = UDPServer((ip_address, 5000), debug)
        self.divider = '!#$'
        self.memoryQueue = list()
        for i in range(0, 10):
            self.memoryQueue.append(-1)

    def recieveMessage(self):
        while True:
            print("\nListening for Client message.\n")
            frame = self.conn.recieveMessage()
            if frame != '':
                splt = frame.split(self.divider)
                n = int(splt[0])
                self.conn.sendMessage(n)
                message = splt[1]
                if n not in self.memoryQueue:
                    self.memoryQueue.insert(0, n)
                    self.memoryQueue.pop()
                    if self.debug:
                        print('DEBUG - {}'.format(self.memoryQueue))
                    return message
                if self.debug:
                    print('DEBUG - Repeated Package')

    def closeConnection(self):
        self.conn.closeConnection()

# Serve Interface.
    - Starts the server and displays any new data.

In [ ]:
exit = False
server = SAWServer('127.0.0.1', True)

while not exit:
    message = server.recieveMessage()
    print("Recieved: " + message)
    if message == "exit":
        exit = True


server.closeConnection()


DEBUG - Starting up on 127.0.0.1 port 5000

Listening for Client message.

DEBUG - waiting to receive message
DEBUG - received 6 bytes from ('127.0.0.1', 54855)
DEBUG - sent 1 bytes back to ('127.0.0.1', 54855)
DEBUG - [0, -1, -1, -1, -1, -1, -1, -1, -1, -1]
Recieved: Hi

Listening for Client message.

DEBUG - waiting to receive message
DEBUG - received 9 bytes from ('127.0.0.1', 54855)
DEBUG - sent 1 bytes back to ('127.0.0.1', 54855)
DEBUG - [1, 0, -1, -1, -1, -1, -1, -1, -1, -1]
Recieved: Hello

Listening for Client message.

DEBUG - waiting to receive message
DEBUG - received 6 bytes from ('127.0.0.1', 54855)
DEBUG - sent 1 bytes back to ('127.0.0.1', 54855)
DEBUG - [2, 1, 0, -1, -1, -1, -1, -1, -1, -1]
Recieved: Yo

Listening for Client message.

DEBUG - waiting to receive message
DEBUG - received 11 bytes from ('127.0.0.1', 54855)
DEBUG - sent 1 bytes back to ('127.0.0.1', 54855)
DEBUG - [3, 2, 1, 0, -1, -1, -1, -1, -1, -1]
Recieved: Bastard

Listening for Client message.

DEBUG 